In [2]:
import numpy as np
import pandas as pd
#from scipy.signal import savgol_filter

df_weather2 = pd.read_csv('/Users/shayan/Desktop/IDS2/Stattkueche/df_weather3.csv',parse_dates=['DateOfCancel','DateOfService'])



In [ ]:
# ###### vif after incorporating the weather varibales for df_encoded
# #### do the vif check
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# num_cols = df_weather2.select_dtypes(include=[np.number]).columns.to_list()
#
# drop_col = ['DateOfOrder', 'DateOfService','days_to_cancel','CanceledQty','OrderId', 'TransactionId','cancel_timing','BookingNr','DateOfCancel','net_qty']
#
# vif_col_1 = [c for c in num_cols if c not in drop_col]
#
# vif_dat_1 = df_encoded[vif_col_1].copy()
# vif_dat_1.replace([np.inf,-np.inf],np.nan,inplace=True)
# vif_dat_1 = vif_dat_1.dropna()
# vif_scores_1 = [variance_inflation_factor(vif_dat_1,i)
#               for i in range(vif_dat_1.shape[1])
#               ]
#
# vif_table_1 = (pd.DataFrame({'feature':vif_col_1,'VIF':vif_scores_1}).sort_values('VIF',ascending=False).reset_index(drop=True))
# print('VIF values')
# display(vif_table_1)
#
# hg_vif_1 = vif_table_1.loc[vif_table_1['VIF']>10,'feature'].to_list()
# print('the high vif columns dropped')
# df_without_vif_1 = vif_dat_1.drop(columns=hg_vif_1)
# print(df_without_vif_1.columns)

In [3]:
### Stage A for random forest with df_weather2

import numpy as np
import pandas as pd
from sklearn.base           import BaseEstimator, TransformerMixin
from sklearn.pipeline       import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster       import KMeans
from sklearn.impute        import SimpleImputer
from sklearn.ensemble      import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from imblearn.pipeline      import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

# ── 1) Prepare X / y for Stage A ────────────────────────────────
df_A    = df_weather2.copy()
# binary target: cancelled if CanceledQty > 0
y_A     = (df_A['CanceledQty'] > 0).astype(int)
X_A     = df_A.drop(columns=[
    'CanceledQty',      # leak
    'cancel_timing',    # multi‐class leftover
    # any other columns you know shouldn’t go in:
    'DateOfOrder','DateOfService','DateOfCancel',
    'OrderId','TransactionId','BookingNr','hist_cancel_rate','GroupName','SchoolID'
])

# ── 2) Re‐use your transformers ───────────────────────────────────
class HistCancelRateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_keys=('Site','MenuBase'), value_col='net_qty', out_col='hist_cancel_rate'):
        self.group_keys = group_keys
        self.value_col  = value_col
        self.out_col    = out_col
    def fit(self, X, y=None):
        keys = list(self.group_keys)
        self.hist_    = X.groupby(keys)[self.value_col].mean()
        self.default_ = self.hist_.median()
        return self
    def transform(self, X):
        keys   = list(self.group_keys)
        tuples = [tuple(r) for r in X[keys].values]
        X      = X.copy()
        X[self.out_col] = [self.hist_.get(t, self.default_) for t in tuples]
        return X

class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, profile_feats, group_keys=('Site','MenuBase'),
                 n_clusters=5, out_col='cluster_id'):
        self.profile_feats = profile_feats
        self.group_keys    = group_keys
        self.n_clusters    = n_clusters
        self.out_col       = out_col
    def fit(self, X, y=None):
        keys = list(self.group_keys)
        prof = (X.groupby(keys)[self.profile_feats].mean().reset_index())
        prof[self.profile_feats] = prof[self.profile_feats].fillna(prof[self.profile_feats].median())
        self.scaler_ = StandardScaler().fit(prof[self.profile_feats])
        scaled      = self.scaler_.transform(prof[self.profile_feats])
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=42).fit(scaled)
        tuples      = [tuple(r) for r in prof[keys].values]
        self.cluster_map_ = dict(zip(tuples, self.kmeans_.labels_))
        self.default_     = int(np.median(self.kmeans_.labels_))
        return self
    def transform(self, X):
        X = X.copy()
        keys   = list(self.group_keys)
        tuples = [tuple(r) for r in X[keys].values]
        X[self.out_col] = [self.cluster_map_.get(t, self.default_) for t in tuples]
        return X

class MissingFlagImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median'):
        self.strategy = strategy
    def fit(self, X, y=None):
        self.num_cols = X.select_dtypes(include=[np.number]).columns
        clean = X[self.num_cols].replace([np.inf,-np.inf], np.nan)
        self.imputer_ = SimpleImputer(strategy=self.strategy).fit(clean)
        return self
    def transform(self, X):
        X = X.copy()
        X[self.num_cols] = X[self.num_cols].replace([np.inf,-np.inf], np.nan)
        for c in self.num_cols:
            X[c + '_missing'] = X[c].isna().astype(int)
        X[self.num_cols] = self.imputer_.transform(X[self.num_cols])
        return X

# ── 3) Drop any remaining non-numeric / ID columns ───────────────
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(columns=self.cols_to_drop, errors='ignore')

drop_cols_A = [
    # any string or ID cols still in X_A
    'Site','MenuBase','MenuName','GroupName','MenuNorm','MenuCode','net_qty','days_to_cancel'
]

# ── 4) Assemble Stage A pipeline ─────────────────────────────────
pipeline_A = ImbPipeline([
    #('te',      TargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),
    ('te',      InCVTargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),


    ('hist',    HistCancelRateTransformer()),
    ('cluster', ClusterTransformer(
                   profile_feats=['hist_cancel_rate','rain_flag','temp_dev','sin_doy', 'cos_doy', 'month',
       'day_of_month', 'is_month_end', 'is_month_start','tavg_C', 'prcp_mm'],
                   n_clusters=5)),
    ('drop',    ColumnDropper(drop_cols_A)),
    ('impute',  MissingFlagImputer()),
    #('smote',   SMOTE(random_state=24)),
    ('clf',     RandomForestClassifier(
                   n_estimators=500,
                   criterion='entropy',     # your experiment
                   max_depth=8,
                   min_samples_split=5,
                   min_samples_leaf=1,
                   max_features='sqrt',
                   class_weight='balanced_subsample',
                   random_state=24,
                   n_jobs=-1
               )),
])

# ── 5) Evaluate with rolling AUC ──────────────────────────────────
tscv    = TimeSeriesSplit(n_splits=5)
scores  = cross_val_score(
    pipeline_A, X_A, y_A,
    cv=tscv,
    scoring='roc_auc',
    n_jobs=-1
)
print("Stage A ROC-AUC:", np.round(scores.mean(),4))



NameError: name 'InCVTargetEncoder' is not defined

In [4]:
### stage A with df_weather2 and lgbm

import numpy as np
import pandas as pd
from sklearn.base           import BaseEstimator, TransformerMixin
from sklearn.pipeline       import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster       import KMeans
from sklearn.impute        import SimpleImputer
from sklearn.ensemble      import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from imblearn.pipeline      import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline as ImbPipeline


# ── 1) Prepare X / y for Stage A ────────────────────────────────
df_A    = df_weather2.copy()
# binary target: cancelled if CanceledQty > 0
y_A     = (df_A['CanceledQty'] > 0).astype(int)
X_A     = df_A.drop(columns=[
    'CanceledQty',      # leak
    'cancel_timing',    # multi‐class leftover
    # any other columns you know shouldn’t go in:
    'DateOfOrder','DateOfService','DateOfCancel',
    'OrderId','TransactionId','BookingNr','hist_cancel_rate','GroupName','SchoolID'
])

# ── 2) Re‐use your transformers ───────────────────────────────────
class HistCancelRateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_keys=('Site','MenuBase'), value_col='net_qty', out_col='hist_cancel_rate'):
        self.group_keys = group_keys
        self.value_col  = value_col
        self.out_col    = out_col
    def fit(self, X, y=None):
        keys = list(self.group_keys)
        self.hist_    = X.groupby(keys)[self.value_col].mean()
        self.default_ = self.hist_.median()
        return self
    def transform(self, X):
        keys   = list(self.group_keys)
        tuples = [tuple(r) for r in X[keys].values]
        X      = X.copy()
        X[self.out_col] = [self.hist_.get(t, self.default_) for t in tuples]
        return X

class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, profile_feats, group_keys=('Site','MenuBase'),
                 n_clusters=5, out_col='cluster_id'):
        self.profile_feats = profile_feats
        self.group_keys    = group_keys
        self.n_clusters    = n_clusters
        self.out_col       = out_col
    def fit(self, X, y=None):
        keys = list(self.group_keys)
        prof = (X.groupby(keys)[self.profile_feats].mean().reset_index())
        prof[self.profile_feats] = prof[self.profile_feats].fillna(prof[self.profile_feats].median())
        self.scaler_ = StandardScaler().fit(prof[self.profile_feats])
        scaled      = self.scaler_.transform(prof[self.profile_feats])
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=42).fit(scaled)
        tuples      = [tuple(r) for r in prof[keys].values]
        self.cluster_map_ = dict(zip(tuples, self.kmeans_.labels_))
        self.default_     = int(np.median(self.kmeans_.labels_))
        return self
    def transform(self, X):
        X = X.copy()
        keys   = list(self.group_keys)
        tuples = [tuple(r) for r in X[keys].values]
        X[self.out_col] = [self.cluster_map_.get(t, self.default_) for t in tuples]
        return X

class MissingFlagImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median'):
        self.strategy = strategy
    def fit(self, X, y=None):
        self.num_cols = X.select_dtypes(include=[np.number]).columns
        clean = X[self.num_cols].replace([np.inf,-np.inf], np.nan)
        self.imputer_ = SimpleImputer(strategy=self.strategy).fit(clean)
        return self
    def transform(self, X):
        X = X.copy()
        X[self.num_cols] = X[self.num_cols].replace([np.inf,-np.inf], np.nan)
        for c in self.num_cols:
            X[c + '_missing'] = X[c].isna().astype(int)
        X[self.num_cols] = self.imputer_.transform(X[self.num_cols])
        return X

# ── 3) Drop any remaining non-numeric / ID columns ───────────────
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(columns=self.cols_to_drop, errors='ignore')

drop_cols_A = [
    # any string or ID cols still in X_A
    'Site','MenuBase','MenuName','GroupName','MenuNorm','MenuCode','net_qty','days_to_cancel'
]

# ── 4) Assemble Stage A pipeline ─────────────────────────────────
pipeline_A_l = ImbPipeline([
    #('te',      TargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),
    ('te',      InCVTargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),


    ('hist',    HistCancelRateTransformer()),
    ('cluster', ClusterTransformer(
                   profile_feats=['hist_cancel_rate','rain_flag','temp_dev','sin_doy', 'cos_doy', 'month',
       'day_of_month', 'is_month_end', 'is_month_start','tavg_C', 'prcp_mm'],
                   n_clusters=5)),
    ('drop',    ColumnDropper(drop_cols_A)),
    ('impute',  MissingFlagImputer()),
    #('smote',   SMOTE(random_state=24)),
    ('clf',       LGBMClassifier(objective='multiclass',
                                 num_class=len(np.unique(y_A)),
                                 random_state=24,
                                 metric    = "multi_logloss",
                                 n_jobs=-1))
,
])
param_dist_lgbm = {
    'clf__n_estimators':      [200,500,800],
    'clf__learning_rate':     [0.01,0.03,0.05],
    'clf__num_leaves':        [31,15,10],
    'clf__max_depth':         [10,6,20],
    'clf__subsample':         [0.7,0.3,1],
    'clf__colsample_bytree':  [1.0,2.0,0.7],
    'clf__min_child_samples': [20,10,7]
}

# 6) Rolling CV search
tscv = TimeSeriesSplit(n_splits=5)
search_A_lgbm = RandomizedSearchCV(
    pipeline_A_l,
    param_distributions=param_dist_lgbm,
    n_iter=30,
    cv=tscv,
    scoring='roc_auc_ovo_weighted',
    n_jobs=-1,
    random_state=24
)

search_A_lgbm.fit(X_A, y_A)

print("Best LGBM ROC_AUC_OVO_weighted:", np.round(search_A_lgbm.best_score_,4))
print("Best hyper‐parameters:")
for k, v in search_A_lgbm.best_params_.items():
    print(f"  {k}: {v}")



NameError: name 'InCVTargetEncoder' is not defined

In [ ]:
#### stage B2 with hist_cancel_rate and cluster calculation inside the cv loop and target encoding
from sklearn.base           import BaseEstimator, TransformerMixin
from sklearn.pipeline       import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster       import KMeans
from sklearn.impute        import SimpleImputer
from sklearn.ensemble      import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder



df_B2 = df_weather2[df_weather2.cancel_timing != 'no_cancel'].copy()
le    = LabelEncoder().fit(df_B2.cancel_timing)
df_B2['timing_code'] = le.transform(df_B2.cancel_timing)

# drop any leak columns before X
X_B2 = df_B2.drop(columns=[
    'CanceledQty',      
    'cancel_timing','timing_code',     
    'DateOfOrder','DateOfService','DateOfCancel',
    'OrderId','TransactionId','BookingNr','hist_cancel_rate','GroupName','SchoolID'
])
y_B2 = df_B2['timing_code']
# 0) Build the Stage B2 subset once:
#df_B2 = df_weather[
#    (df_weather.CanceledQty > 0) &
#    (df_weather.cancel_timing.isin([0,1,2]))
#].copy()
#X_full = df_B2.drop(columns='cancel_timing')
#y_full = df_B2['cancel_timing']

# Update to HistCancelRateTransformer and ClusterTransformer to avoid tuple KeyError

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class InCVTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols, smoothing=1.0):
        self.cols      = cols
        self.smoothing = smoothing

    def fit(self, X, y):
        self.global_mean_ = y.mean()
        self.mapping_ = {}
        for c in self.cols:
            df = pd.DataFrame({c: X[c], 'target': y})
            agg = df.groupby(c)['target'].agg(['mean','count'])
            # smoothing formula
            agg['enc'] = (
                (agg['count'] * agg['mean'] + 
                 self.smoothing * self.global_mean_)
                / (agg['count'] + self.smoothing)
            )
            self.mapping_[c] = agg['enc']
        return self

    def transform(self, X):
        X = X.copy()
        for c in self.cols:
            X[c + '_te'] = X[c]\
                .map(self.mapping_[c])\
                .fillna(self.global_mean_)
        return X



class HistCancelRateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_keys=('Site','MenuBase'), value_col='net_qty', out_col='hist_cancel_rate'):
        # Store parameters verbatim—do NOT convert to list here
        self.group_keys = group_keys
        self.value_col  = value_col
        self.out_col    = out_col
        
    def fit(self, X, y=None):
        # Convert to list only when you need it
        keys = list(self.group_keys)
        self.hist_    = X.groupby(keys)[self.value_col].mean()
        self.default_ = self.hist_.median()
        return self
    
    def transform(self, X):
        keys = list(self.group_keys)
        # build tuple keys and map
        tuples = list(zip(*(X[k] for k in keys)))
        X = X.copy()
        X[self.out_col] = [self.hist_.get(t, self.default_) for t in tuples]
        return X


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        profile_feats,                 # list of column names to form profiles
        group_keys=('Site','MenuBase'),
        n_clusters=5,
        out_col='cluster_id'
    ):
        # Store parameters as‐is; don’t mutate them here
        self.profile_feats = profile_feats
        self.group_keys    = group_keys
        self.n_clusters    = n_clusters
        self.out_col       = out_col

    def fit(self, X, y=None):
        # 1) Build training set profiles by group
        keys = list(self.group_keys)
        prof = (
            X.groupby(keys)[self.profile_feats]
             .mean()
             .reset_index()
        )
        # 2) Median‐impute any missing values in the profiles
        prof[self.profile_feats] = prof[self.profile_feats].fillna(
            prof[self.profile_feats].median()
        )

        # 3) Standardize and run K-Means on those profiles
        self.scaler_ = StandardScaler().fit(prof[self.profile_feats])
        scaled      = self.scaler_.transform(prof[self.profile_feats])
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=42).fit(scaled)

        # 4) Build a lookup map from (Site,MenuBase) → cluster label
        tuples = [tuple(row) for row in prof[keys].values]
        labels = list(self.kmeans_.labels_)
        self.cluster_map_ = dict(zip(tuples, labels))
        # Use the median cluster as a fallback for unseen combos
        self.default_ = int(np.median(labels))
        return self

    def transform(self, X):
        X = X.copy()
        keys   = list(self.group_keys)
        tuples = [tuple(row) for row in X[keys].values]
        # Map each row’s key to its cluster (or default)
        X[self.out_col] = [
            self.cluster_map_.get(t, self.default_) for t in tuples
        ]
        return X



class MissingFlagImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median'):
        self.strategy = strategy

    def fit(self, X, y=None):
        # Identify numeric columns
        self.num_cols = X.select_dtypes(include=[np.number]).columns
        # Replace inf with NaN for fitting
        clean = X[self.num_cols].replace([np.inf, -np.inf], np.nan)
        # Fit the imputer on cleaned data
        self.imputer_ = SimpleImputer(strategy=self.strategy)
        self.imputer_.fit(clean)
        return self

    def transform(self, X):
        X = X.copy()
        # Replace inf with NaN before transforming
        X[self.num_cols] = X[self.num_cols].replace([np.inf, -np.inf], np.nan)
        # Add missing flags
        for c in self.num_cols:
            X[c + '_missing'] = X[c].isna().astype(int)
        # Impute
        X[self.num_cols] = self.imputer_.transform(X[self.num_cols])
        return X

# Example integration into your pipeline:
# pipeline_B2 = Pipeline([
#     ('hist', HistCancelRateTransformer()),
#     ('cluster', ClusterTransformer(...)),
#     ('impute', MissingFlagImputer()),
#     ('clf', RandomForestClassifier(...))
# ])

# 4) Assemble pipelin
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Drop any columns we know are non-numeric, IDs, or leftovers
        return X.drop(columns=self.cols_to_drop,errors='ignore')
drop_cols_B2 = [
    'MenuName','GroupName','MenuNorm','MenuCode'

]
drop_post = ['Site','MenuBase','net_qty','days_to_cancel']
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
# 4) Assemble pipelin

#pipeline_B2 = Pipeline([
#    ('drop',    ColumnDropper(drop_cols_B2)),
#    ('hist',    HistCancelRateTransformer()),         # train-only hist rates
#    ('cluster', ClusterTransformer(
#                   profile_feats=['hist_cancel_rate','rain_flag','temp_dev'],
#                   n_clusters=5)),
#    ('impute',  MissingFlagImputer()),               # replace inf → NaN, flag + median
#    ('clf',     RandomForestClassifier(
#                   n_estimators=100,
#                   class_weight='balanced',
#                   random_state=24,
#                   n_jobs=-1
#               )),
#])

#print("Columns going into the pipeline:", X_full.columns.tolist())
pipeline_B2_smote = ImbPipeline([
    ('te',      InCVTargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),
    ('drop',    ColumnDropper(drop_cols_B2)),                # drop strings, IDs, leakage cols
    ('hist',    HistCancelRateTransformer()),                # train‐only hist rates
    ('cluster', ClusterTransformer(                          # train‐only clusters
                    profile_feats=['hist_cancel_rate','rain_flag','temp_dev','sin_doy', 'cos_doy', 'month',
       'day_of_month', 'is_month_end', 'is_month_start','tavg_C', 'prcp_mm'],
                    n_clusters=5)),
    ('drop_post', ColumnDropper(drop_post)),               
    ('impute',  MissingFlagImputer()),                       # inf→nan, flag & median‐impute
    ('smote',   SMOTE(random_state=24)),                     # synthesize new minority samples
    ('clf',     RandomForestClassifier(                      # your tuned RF head
                   n_estimators=500,
                   min_samples_split=5,
                   min_samples_leaf=2,
                   max_features='sqrt',
                   max_depth=None,
                   class_weight='balanced',
                   random_state=24,
                   n_jobs=-1
               )),
])
# 5) Rolling CV evaluation
tscv    = TimeSeriesSplit(n_splits=5)
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import numpy as np

# 1) Define the time‐series split
#tscv = TimeSeriesSplit(n_splits=5)

# 2) Specify the RF hyper‐parameter distributions
param_dist = {
    'clf__n_estimators':      [600,1200],
    'clf__max_depth':         [10,6],
    'clf__min_samples_split': [10,20],
    'clf__min_samples_leaf':  [15],
    'clf__bootstrap':         [True],
    'clf__criterion':         ['entropy'],
    'clf__max_features':      [0.8,1.2]
}

search = RandomizedSearchCV(
    pipeline_B2_smote,               # the full hist→cluster→impute→RF pipe
    param_distributions=param_dist,
    n_iter=20,                 # how many random combos to try
    cv=tscv,                   # your time‐series folds
    scoring='roc_auc_ovo_weighted',  # optimize roc_auc_ovo_weighted
    n_jobs=-1,
    random_state=24
)

# 4) Fit on your Stage B2 data
search.fit(X_B2, y_B2)

# 5) Check the results
print("Best roc_auc_ovo_weighted:", np.round(search.best_score_, 4))
print("Best hyper-parameters:")
for k, v in search.best_params_.items():
    print(f"  {k} = {v}")
#scores  = cross_val_score(
#    pipeline_B2,
#    X_full,
#    y_full,
#    cv=tscv,
#    scoring='average_precision',
#    n_jobs=1
#)
#print("Stage B2 PR-AUC:", np.round(scores.mean(),4))


In [ ]:
#### stage B2 with hist_cancel_rate and cluster calculation inside the cv loop and target encoding LGBM
from sklearn.base           import BaseEstimator, TransformerMixin
from sklearn.pipeline       import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster       import KMeans
from sklearn.impute        import SimpleImputer
from sklearn.ensemble      import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier




df_B2 = df_weather2[df_weather2.cancel_timing != 'no_cancel'].copy()
le    = LabelEncoder().fit(df_B2.cancel_timing)
df_B2['timing_code'] = le.transform(df_B2.cancel_timing)

# drop any leak columns before X
X_B2 = df_B2.drop(columns=[
    'CanceledQty',      
    'cancel_timing','timing_code',     
    'DateOfOrder','DateOfService','DateOfCancel',
    'OrderId','TransactionId','BookingNr','hist_cancel_rate','GroupName','SchoolID'
])
y_B2 = df_B2['timing_code']
# 0) Build the Stage B2 subset once:
#df_B2 = df_weather[
#    (df_weather.CanceledQty > 0) &
#    (df_weather.cancel_timing.isin([0,1,2]))
#].copy()
#X_full = df_B2.drop(columns='cancel_timing')
#y_full = df_B2['cancel_timing']

# Update to HistCancelRateTransformer and ClusterTransformer to avoid tuple KeyError

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class InCVTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols, smoothing=1.0):
        self.cols      = cols
        self.smoothing = smoothing

    def fit(self, X, y):
        self.global_mean_ = y.mean()
        self.mapping_ = {}
        for c in self.cols:
            df = pd.DataFrame({c: X[c], 'target': y})
            agg = df.groupby(c)['target'].agg(['mean','count'])
            # smoothing formula
            agg['enc'] = (
                (agg['count'] * agg['mean'] + 
                 self.smoothing * self.global_mean_)
                / (agg['count'] + self.smoothing)
            )
            self.mapping_[c] = agg['enc']
        return self

    def transform(self, X):
        X = X.copy()
        for c in self.cols:
            X[c + '_te'] = X[c]\
                .map(self.mapping_[c])\
                .fillna(self.global_mean_)
        return X



class HistCancelRateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_keys=('Site','MenuBase'), value_col='net_qty', out_col='hist_cancel_rate'):
        # Store parameters verbatim—do NOT convert to list here
        self.group_keys = group_keys
        self.value_col  = value_col
        self.out_col    = out_col
        
    def fit(self, X, y=None):
        # Convert to list only when you need it
        keys = list(self.group_keys)
        self.hist_    = X.groupby(keys)[self.value_col].mean()
        self.default_ = self.hist_.median()
        return self
    
    def transform(self, X):
        keys = list(self.group_keys)
        # build tuple keys and map
        tuples = list(zip(*(X[k] for k in keys)))
        X = X.copy()
        X[self.out_col] = [self.hist_.get(t, self.default_) for t in tuples]
        return X


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        profile_feats,                 # list of column names to form profiles
        group_keys=('Site','MenuBase'),
        n_clusters=5,
        out_col='cluster_id'
    ):
        # Store parameters as‐is; don’t mutate them here
        self.profile_feats = profile_feats
        self.group_keys    = group_keys
        self.n_clusters    = n_clusters
        self.out_col       = out_col

    def fit(self, X, y=None):
        # 1) Build training set profiles by group
        keys = list(self.group_keys)
        prof = (
            X.groupby(keys)[self.profile_feats]
             .mean()
             .reset_index()
        )
        # 2) Median‐impute any missing values in the profiles
        prof[self.profile_feats] = prof[self.profile_feats].fillna(
            prof[self.profile_feats].median()
        )

        # 3) Standardize and run K-Means on those profiles
        self.scaler_ = StandardScaler().fit(prof[self.profile_feats])
        scaled      = self.scaler_.transform(prof[self.profile_feats])
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=42).fit(scaled)

        # 4) Build a lookup map from (Site,MenuBase) → cluster label
        tuples = [tuple(row) for row in prof[keys].values]
        labels = list(self.kmeans_.labels_)
        self.cluster_map_ = dict(zip(tuples, labels))
        # Use the median cluster as a fallback for unseen combos
        self.default_ = int(np.median(labels))
        return self

    def transform(self, X):
        X = X.copy()
        keys   = list(self.group_keys)
        tuples = [tuple(row) for row in X[keys].values]
        # Map each row’s key to its cluster (or default)
        X[self.out_col] = [
            self.cluster_map_.get(t, self.default_) for t in tuples
        ]
        return X



class MissingFlagImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median'):
        self.strategy = strategy

    def fit(self, X, y=None):
        # Identify numeric columns
        self.num_cols = X.select_dtypes(include=[np.number]).columns
        # Replace inf with NaN for fitting
        clean = X[self.num_cols].replace([np.inf, -np.inf], np.nan)
        # Fit the imputer on cleaned data
        self.imputer_ = SimpleImputer(strategy=self.strategy)
        self.imputer_.fit(clean)
        return self

    def transform(self, X):
        X = X.copy()
        # Replace inf with NaN before transforming
        X[self.num_cols] = X[self.num_cols].replace([np.inf, -np.inf], np.nan)
        # Add missing flags
        for c in self.num_cols:
            X[c + '_missing'] = X[c].isna().astype(int)
        # Impute
        X[self.num_cols] = self.imputer_.transform(X[self.num_cols])
        return X

# Example integration into your pipeline:
# pipeline_B2 = Pipeline([
#     ('hist', HistCancelRateTransformer()),
#     ('cluster', ClusterTransformer(...)),
#     ('impute', MissingFlagImputer()),
#     ('clf', RandomForestClassifier(...))
# ])

# 4) Assemble pipelin
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Drop any columns we know are non-numeric, IDs, or leftovers
        return X.drop(columns=self.cols_to_drop,errors='ignore')
drop_cols_B2 = [
    'MenuName','GroupName','MenuNorm','MenuCode'

]
drop_post = ['Site','MenuBase','net_qty','days_to_cancel']
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
# 4) Assemble pipelin

#pipeline_B2 = Pipeline([
#    ('drop',    ColumnDropper(drop_cols_B2)),
#    ('hist',    HistCancelRateTransformer()),         # train-only hist rates
#    ('cluster', ClusterTransformer(
#                   profile_feats=['hist_cancel_rate','rain_flag','temp_dev'],
#                   n_clusters=5)),
#    ('impute',  MissingFlagImputer()),               # replace inf → NaN, flag + median
#    ('clf',     RandomForestClassifier(
#                   n_estimators=100,
#                   class_weight='balanced',
#                   random_state=24,
#                   n_jobs=-1
#               )),
#])

#print("Columns going into the pipeline:", X_full.columns.tolist())
pipeline_B2_lgbm = ImbPipeline([
    ('te',      InCVTargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),
    ('drop',    ColumnDropper(drop_cols_B2)),                # drop strings, IDs, leakage cols
    ('hist',    HistCancelRateTransformer()),                # train‐only hist rates
    ('cluster', ClusterTransformer(                          # train‐only clusters
                    profile_feats=['hist_cancel_rate','rain_flag','temp_dev','sin_doy', 'cos_doy', 'month',
       'day_of_month', 'is_month_end', 'is_month_start','tavg_C', 'prcp_mm'],
                    n_clusters=5)),
    ('drop_post', ColumnDropper(drop_post)),               
    ('impute',  MissingFlagImputer()),                       # inf→nan, flag & median‐impute
    #('smote',   SMOTE(random_state=24)),                     # synthesize new minority samples
   ('clf',       LGBMClassifier(objective='multiclass',
                                 num_class=len(np.unique(y_B2)),
                                 random_state=24,
                                 metric    = "multi_logloss",
                                 n_jobs=-1))
,
])
param_dist_lgbm = {
    'clf__n_estimators':      [200,600],
    'clf__learning_rate':     [0.01,0.07],
    'clf__num_leaves':        [31,40],
    'clf__max_depth':         [10,20],
    'clf__subsample':         [0.7,1],
    'clf__colsample_bytree':  [1.0,2],
    'clf__min_child_samples': [20,30]
}

# 6) Rolling CV search
tscv = TimeSeriesSplit(n_splits=5)
search_lgbm = RandomizedSearchCV(
    pipeline_B2_lgbm,
    param_distributions=param_dist_lgbm,
    n_iter=30,
    cv=tscv,
    scoring='roc_auc_ovo_weighted',
    n_jobs=-1,
    random_state=24
)

search_lgbm.fit(X_B2, y_B2)

print("Best LGBM ROC_AUC_OVO_weighted:", np.round(search_lgbm.best_score_,4))
print("Best hyper‐parameters:")
for k, v in search_lgbm.best_params_.items():
    print(f"  {k}: {v}")


In [ ]:
#### stage B2 with hist_cancel_rate and cluster calculation inside the cv loop and target encoding catboost
from sklearn.base           import BaseEstimator, TransformerMixin
from sklearn.pipeline       import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster       import KMeans
from sklearn.impute        import SimpleImputer
from sklearn.ensemble      import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import make_scorer, f1_score, balanced_accuracy_score




df_B2 = df_weather2[df_weather2.cancel_timing != 'no_cancel'].copy()
le    = LabelEncoder().fit(df_B2.cancel_timing)
df_B2['timing_code'] = le.transform(df_B2.cancel_timing)

# drop any leak columns before X
X_B2 = df_B2.drop(columns=[
    'CanceledQty',      
    'cancel_timing','timing_code',     
    'DateOfOrder','DateOfService','DateOfCancel',
    'OrderId','TransactionId','BookingNr','hist_cancel_rate','GroupName','SchoolID'
])
y_B2 = df_B2['timing_code']
# 0) Build the Stage B2 subset once:
#df_B2 = df_weather[
#    (df_weather.CanceledQty > 0) &
#    (df_weather.cancel_timing.isin([0,1,2]))
#].copy()
#X_full = df_B2.drop(columns='cancel_timing')
#y_full = df_B2['cancel_timing']

# Update to HistCancelRateTransformer and ClusterTransformer to avoid tuple KeyError

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class InCVTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols, smoothing=1.0):
        self.cols      = cols
        self.smoothing = smoothing

    def fit(self, X, y):
        self.global_mean_ = y.mean()
        self.mapping_ = {}
        for c in self.cols:
            df = pd.DataFrame({c: X[c], 'target': y})
            agg = df.groupby(c)['target'].agg(['mean','count'])
            # smoothing formula
            agg['enc'] = (
                (agg['count'] * agg['mean'] + 
                 self.smoothing * self.global_mean_)
                / (agg['count'] + self.smoothing)
            )
            self.mapping_[c] = agg['enc']
        return self

    def transform(self, X):
        X = X.copy()
        for c in self.cols:
            X[c + '_te'] = X[c]\
                .map(self.mapping_[c])\
                .fillna(self.global_mean_)
        return X



class HistCancelRateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_keys=('Site','MenuBase'), value_col='net_qty', out_col='hist_cancel_rate'):
        # Store parameters verbatim—do NOT convert to list here
        self.group_keys = group_keys
        self.value_col  = value_col
        self.out_col    = out_col
        
    def fit(self, X, y=None):
        # Convert to list only when you need it
        keys = list(self.group_keys)
        self.hist_    = X.groupby(keys)[self.value_col].mean()
        self.default_ = self.hist_.median()
        return self
    
    def transform(self, X):
        keys = list(self.group_keys)
        # build tuple keys and map
        tuples = list(zip(*(X[k] for k in keys)))
        X = X.copy()
        X[self.out_col] = [self.hist_.get(t, self.default_) for t in tuples]
        return X


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        profile_feats,                 # list of column names to form profiles
        group_keys=('Site','MenuBase'),
        n_clusters=5,
        out_col='cluster_id'
    ):
        # Store parameters as‐is; don’t mutate them here
        self.profile_feats = profile_feats
        self.group_keys    = group_keys
        self.n_clusters    = n_clusters
        self.out_col       = out_col

    def fit(self, X, y=None):
        # 1) Build training set profiles by group
        keys = list(self.group_keys)
        prof = (
            X.groupby(keys)[self.profile_feats]
             .mean()
             .reset_index()
        )
        # 2) Median‐impute any missing values in the profiles
        prof[self.profile_feats] = prof[self.profile_feats].fillna(
            prof[self.profile_feats].median()
        )

        # 3) Standardize and run K-Means on those profiles
        self.scaler_ = StandardScaler().fit(prof[self.profile_feats])
        scaled      = self.scaler_.transform(prof[self.profile_feats])
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=42).fit(scaled)

        # 4) Build a lookup map from (Site,MenuBase) → cluster label
        tuples = [tuple(row) for row in prof[keys].values]
        labels = list(self.kmeans_.labels_)
        self.cluster_map_ = dict(zip(tuples, labels))
        # Use the median cluster as a fallback for unseen combos
        self.default_ = int(np.median(labels))
        return self

    def transform(self, X):
        X = X.copy()
        keys   = list(self.group_keys)
        tuples = [tuple(row) for row in X[keys].values]
        # Map each row’s key to its cluster (or default)
        X[self.out_col] = [
            self.cluster_map_.get(t, self.default_) for t in tuples
        ]
        return X



class MissingFlagImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median'):
        self.strategy = strategy

    def fit(self, X, y=None):
        # Identify numeric columns
        self.num_cols = X.select_dtypes(include=[np.number]).columns
        # Replace inf with NaN for fitting
        clean = X[self.num_cols].replace([np.inf, -np.inf], np.nan)
        # Fit the imputer on cleaned data
        self.imputer_ = SimpleImputer(strategy=self.strategy)
        self.imputer_.fit(clean)
        return self

    def transform(self, X):
        X = X.copy()
        # Replace inf with NaN before transforming
        X[self.num_cols] = X[self.num_cols].replace([np.inf, -np.inf], np.nan)
        # Add missing flags
        for c in self.num_cols:
            X[c + '_missing'] = X[c].isna().astype(int)
        # Impute
        X[self.num_cols] = self.imputer_.transform(X[self.num_cols])
        return X

# Example integration into your pipeline:
# pipeline_B2 = Pipeline([
#     ('hist', HistCancelRateTransformer()),
#     ('cluster', ClusterTransformer(...)),
#     ('impute', MissingFlagImputer()),
#     ('clf', RandomForestClassifier(...))
# ])

# 4) Assemble pipelin
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Drop any columns we know are non-numeric, IDs, or leftovers
        return X.drop(columns=self.cols_to_drop,errors='ignore')
drop_cols_B2 = [
    'MenuName','GroupName','MenuNorm','MenuCode'

]
drop_post = ['Site','MenuBase','net_qty','days_to_cancel']
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
# 4) Assemble pipelin

#pipeline_B2 = Pipeline([
#    ('drop',    ColumnDropper(drop_cols_B2)),
#    ('hist',    HistCancelRateTransformer()),         # train-only hist rates
#    ('cluster', ClusterTransformer(
#                   profile_feats=['hist_cancel_rate','rain_flag','temp_dev'],
#                   n_clusters=5)),
#    ('impute',  MissingFlagImputer()),               # replace inf → NaN, flag + median
#    ('clf',     RandomForestClassifier(
#                   n_estimators=100,
#                   class_weight='balanced',
#                   random_state=24,
#                   n_jobs=-1
#               )),
#])

#print("Columns going into the pipeline:", X_full.columns.tolist())
pipeline_B2_cat = ImbPipeline([
    ('te',      InCVTargetEncoder(cols=['Site','MenuBase'], smoothing=0.3)),
    ('drop',    ColumnDropper(drop_cols_B2)),                # drop strings, IDs, leakage cols
    ('hist',    HistCancelRateTransformer()),                # train‐only hist rates
    ('cluster', ClusterTransformer(                          # train‐only clusters
                    profile_feats=['hist_cancel_rate','rain_flag','temp_dev','sin_doy', 'cos_doy', 'month',
       'day_of_month', 'is_month_end', 'is_month_start','tavg_C', 'prcp_mm'],
                    n_clusters=5)),
    ('drop_post', ColumnDropper(drop_post)),               
    ('impute',  MissingFlagImputer()),                       # inf→nan, flag & median‐impute
    #('smote',   SMOTE(random_state=24)),                     # synthesize new minority samples
    ('clf',     CatBoostClassifier(
                    # safe defaults:
                    iterations=500,
                    #cat_features = cat_col,
                    auto_class_weights='Balanced',
                    loss_function='MultiClass',
                    #use_focal_loss=True,
                    #focal_loss_gamma=2,
                    #loss_function='Focal',       # turn focal loss ON
                    #focal_loss_gamma=2.0,
                    learning_rate=0.05,
                    depth=6,
                    early_stopping_rounds=50,
                    l2_leaf_reg=3,
                    # avoid verbose logs
                    verbose=False,
                    random_seed=24,
                    thread_count=-1
                    
               ))
])

# 3) CatBoost hyperparameter grid
cat_param_dist = {
    'clf__iterations':       [500,1000,800],
    'clf__loss_function':      ['MultiClass'],
    #'clf__use_focal_loss':     [True, False],       # try both
    #'clf__focal_loss_gamma':   [1.0, 2.0, 5.0], 
    #'clf__focal_alpha':        [0.25, 1.0, 2.0], 
    'clf__learning_rate':    [0.01, 0.03, 0.1],
    'clf__depth':            [6,8,10],
    'clf__l2_leaf_reg':      [1,3,10],
    'clf__bagging_temperature': [0,3,7],
    'clf__rsm':              [0.5, 0.8, 1.0]
}
scoring_1 = {
    'pr_auc':       'average_precision',       # your primary metric
    'roc_auc_ovo':  'roc_auc_ovo',             # multiclass ROC‐AUC
    'f1_macro':     'f1_macro',                # overall F1 across classes
    'bal_acc':      make_scorer(balanced_accuracy_score),
    'neg_log_loss': 'neg_log_loss'             # lower log‐loss = better
}

# 6) Rolling CV search
tscv = TimeSeriesSplit(n_splits=5)
search_cat = RandomizedSearchCV(
    pipeline_B2_cat,
    param_distributions=cat_param_dist,
    n_iter=30,
    cv=tscv,
    refit='roc_auc_ovo',
    scoring=scoring_1,
    n_jobs=-1,
    random_state=24
)

search_cat.fit(X_B2, y_B2)

print("Best LGBM ROC_AUC_OVO_weighted:", np.round(search_cat.best_score_,4))
print("Best hyper‐parameters:")
for k, v in search_cat.best_params_.items():
    print(f"  {k}: {v}")
